# Regressão Linear Múltipla  
---  

<font color='cyan'>author</font> João Baiochi  
<font color='cyan'>github</font> <a href='https://github.com/baiochi'>@baiochi</a>

In [ ]:
# Data handling
import numpy as np
import pandas as pd
# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Statistics
from scipy.stats import normaltest, pearsonr

# Machine Learning
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split

# terminal colors
WHITE = '\033[39m'
CYAN = '\033[36m'
GREEN = '\033[32m'
RED = '\033[31m'

# color pallete
colors ={
    'cyan': '#1696d2',
    'gray': '#5c5859',
    'black': '#000000',
    'yellow': '#fdbf11',
    'orange': '#ca5800',
    'magenta': '#af1f6b',
    'green': '#408941',
    'red': '#a4201d'
}

# setups
sns.set()

# libraries version
print(f'Numpy: {np.__version__}')
print(f'Pandas: {pd.__version__}')
print(f'Seaborn: {sns.__version__}')
print(f'Statsmodels.api: {sm.__version__}')

## 1. Previsão do Saldo Médio do Cartão de Crédito

Neste case, você deverá aplicar a regressão linear múltipla para **estimar o saldo médido** do cartão de crédito de clientes de um banco, a partir das demais informações contidas no conjunto de dados (credit_regression.csv), os quais são descritos abaixo. Aplique a regressão linear, utilizando o LinearRegression do sklearn e, também, o OLS do statsmodels.  

- ID - Identificação  
- Income - Renda  
- Limit - Limite de crédito  
- Rating - Score do Serasa  
- Age - Idade  
- Education - Anos de educação  
- Gender - Gênero  
- Student - É estudante? (Sim ou Não)  
- Married - É casado(a)? (Sim ou Não)  
- Ethnicity - Afro-americano, asiático ou branco  
- Balance - Saldo médio do cartão de crédito em dólares  


Dados adicionais  

- Credit card Balance refers to the average monthly balance across all of the cards owned by a cardholder. 
This assumption was made as a result of the Cards variable which refers to the number of credit cards owned by a person and has only one associated Balance figure.  
- The Balance is calculated as the highest amount incurred on a credit card in a given month. 
For example if a cardholder spends $400, $500, and $600 over the course of three months, and each month pays the balance in full, the average balance will be recorded as $500 (i.e. any preliminary balances before the maximum are not taken into account, neither is the final balance of zero).  

### <span style='color:Gold'>Analisando o Dataset

#### Leitura e verificação do Dataset

In [ ]:
df = pd.read_csv('https://s3-sa-east-1.amazonaws.com/lcpi/ab0a9e91-89d1-4897-b384-51a86c50b804.csv')
df.head()

Verificando os valores da primeira coluna (ID)

In [ ]:
df['Unnamed: 0'].nunique(), df['Unnamed: 0'].min(), df['Unnamed: 0'].max()

Transformando a coluna ID do Data Frame para o index

In [ ]:
df.rename(columns={'Unnamed: 0': 'ID'}, inplace=True)
df.set_index('ID', drop=True, inplace=True)
df.head()

Verificando os tipos dos dados

In [ ]:
df.dtypes

Alterando o tipo das variáveis categóricas para **category**

In [ ]:
df.Gender = df.Gender.astype('category')
df.Student = df.Student.astype('category')
df.Married = df.Married.astype('category')
df.Ethnicity = df.Ethnicity.astype('category')

Separando as variáveis em numéricas e categóricas

In [ ]:
categorical_data = df.select_dtypes(include=['category']).columns
numerical_data = df.select_dtypes(include=['int64','float64']).columns
print(f'Variáveis numéricas: {numerical_data.values}')
print(f'Variáveis categóricas: {categorical_data.values}')

Checando os valores categóricos

In [ ]:
df.describe(include=['category']).T

Checando os valores categóricos

In [ ]:
df.describe().T

Checando valores nulos

In [ ]:
df.isnull().sum()

#### Analisando a variável da predição <span style='color:DodgerBlue'>Balance</span>

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(12,6))
sns.histplot(data=df, x='Balance', kde=True, ax=ax[0], color=colors['cyan'])
ax[0].set_title('Balance (bins=auto)', fontsize=18, pad=15)
sns.histplot(data=df, x='Balance', kde=True, bins=50, ax=ax[1], color=colors['gray'])
ax[1].set_title('Balance (bins=50)', fontsize=18, pad=15);

Visualizando os dados para valores de **Balance** > 0

In [ ]:
sns.histplot(data=df.query('Balance > 0'), x='Balance', kde=True, color=colors['cyan'])
plt.title('Balance > 0 distribuition (bins=auto)', fontsize=18, pad=20);

In [ ]:
print(f'Balance = 0 : {df.query("Balance == 0").shape[0]}')
print(f'Balance > 0 : {df.query("Balance > 0").shape[0]}')

Podemos observar que muitos dados da coluna **Balance** estão com valores zerados. Isso pode significar que o usuário não utiliza o cartão de crédito, e isso pode prejudicar nosso modelo.

Portanto iremos adicionar uma coluna **Credit_Client** que identifica se o usuário faz uso do cartão:

In [ ]:
df['Credit_Client'] = np.where(df['Balance']>0, 'Yes', 'No')
df['Credit_Client'] = df['Credit_Client'].astype('category')
df.head()

#### Correlação dos Dados

Plotando um Heatmap para visualizar a correlação

In [ ]:
plt.figure(figsize=(8,6))
sns.heatmap(df.corr(), cmap='coolwarm', annot=True, vmin=-1, vmax=1);

Checando se a correlação tem significância estatística:
- Balance x Income = 0.46
- Balance x Limit = 0.86
- Balance x Rating = 0.86
- Income x Limit = 0.79
- Income x Rating = 0.79
- Limit X Rating = 1

In [ ]:
for category in ['Income','Limit', 'Rating']:
    r, p = pearsonr(df.Balance, df[category])
    print(f'Balance-{category} coef={r}; p-value={p}')

for category in ['Limit', 'Rating']:
    r, p = pearsonr(df.Income, df[category])
    print(f'Income-{category} coef={r}; p-value={p}')
    
r, p = pearsonr(df.Limit, df.Rating)
print(f'Limit-Rating coef={r}; p-value={p}')

In [ ]:
sns.regplot(data=df, x='Limit', y='Rating',
           scatter_kws={'alpha':0.5}, line_kws={'color':colors['gray']})

In [ ]:
f, axes = plt.subplots(2, 2, figsize=(15, 6))
f.subplots_adjust(hspace=.3, wspace=.25)
df.groupby('Gender').Balance.plot(kind='kde', ax=axes[0][0], legend=True, title='Balance by Gender')
df.groupby('Student').Balance.plot(kind='kde', ax=axes[0][1], legend=True, title='Balance by Student')
df.groupby('Married').Balance.plot(kind='kde', ax=axes[1][0], legend=True, title='Balance by Married')
df.groupby('Ethnicity').Balance.plot(kind='kde', ax=axes[1][1], legend=True, title='Balance by Ethnicity')

#### Pair Plot

In [ ]:
sns.pairplot(df, hue='Student')

#### Boxplot

In [ ]:
sns.boxplot(data=df[numerical_data])

In [ ]:
sns.boxplot(data=df.select_dtypes(include=['int64']))

#### Correlation Plot

In [ ]:
df.corr()

### <span style='color:Gold'>Aplicando a Regressão Linear Multipla

In [ ]:
def create_model_LR(df, features, predict):
    # Set variables
    if predict in features:
        X = df[features].drop(predict, axis=1)
        print(X.head())
    else:
        X = df[features]
        
    y = df[predict]
    # Train model
    lr = LinearRegression().fit(X,y)
    # Predict model
    y_est = lr.predict(X)
    # Results
    print('Coef: ', lr.coef_)
    print('Intercept: ', lr.intercept_)
    print('Score: ', lr.score(X,y))

    return lr


def create_model_SM(df, predict):
    # Set variables
    X = df.drop(predict, axis=1)
    y = df[predict]

    x = sm.add_constant(X)
    model = sm.OLS(y, x).fit()
    print(model.summary())
    print(f'Model mean: {model.resid.mean()}')

    fig, axes = plt.subplots(1,2,figsize=(12,6))
    sns.scatterplot(y, model.resid, ax=axes[0])
    sns.histplot(model2.resid, kde=True, ax=axes[1])

    return model


In [ ]:
df2 = df.copy()

df2 = df2[df2.Balance!=0]

In [ ]:
df2_dummies = pd.get_dummies(df2)

In [ ]:
model = create_model_SM(df2_dummies, 'Balance')


In [ ]:
model2 = create_model_SM(df2_dummies.drop('Education', axis=1), 'Balance')

In [ ]:
normaltest(model2.resid)[1]

In [ ]:
sns.jointplot(data=df2_dummies, x='Limit', y='Balance', kind='reg')

In [ ]:
df_log = df2_dummies.copy()

In [ ]:
df_log[numerical_data] = df_log[numerical_data].apply(lambda x : np.log(x))

In [ ]:
X_final = df_log.drop('Balance', axis=1)
y_final = df_log.Balance

X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.3, random_state=101)

In [ ]:
lr = LinearRegression().fit(X_train, y_train)
y_pred = np.exp(lr.predict(X_test))
y_true = np.exp(y_test)
r2_score(y_true, y_pred)

In [ ]:
lr.score(X_train, y_train)

#### Preparando as variáveis
X = Limit, Rating, Cards, Age, Education  
y = Balance  

In [ ]:
X = df[numerical_data].drop('Balance',axis=1)
X.head()

In [ ]:
y = df['Balance']
y.head()

#### LinearRegression

In [ ]:
# Train model
lr_inital = LinearRegression().fit(X,y)
# Predict model
y_est_initial = lr_inital.predict(X)

In [ ]:
lr_inital.coef_ , lr_inital.intercept_

In [ ]:
lr_inital.score(X,y)

#### Statsmodels

In [ ]:
x = sm.add_constant(X)

model = sm.OLS(y, x).fit()

model.summary()

#### Analisando o Erro da Regressão

In [ ]:
model.resid
plt.plot(model.resid)

In [ ]:
model.resid.mean()

In [ ]:
sns.histplot(model.resid, kde=True)

In [ ]:
plt.scatter(y, model.resid)

## 2. Estimando o Preço de Venda de Casas


O arquivo usa_housing.csv consiste em um dataset que contém informações sobre o preço de casas em determinadas regiões dos Estados Unidos. Uma descrição das colunas desse dataframes é apresentada abaixo:  
  
- Avg. Area Income: Média da renda dos residentes de onde a casa está localizada.  
- Avg. Area House Age: Média de idade das casas da mesma cidade.  
- Avg. Area Number of Rooms: Número médio de quartos para casas na mesma cidade.  
- Avg. Area Number of Bedrooms: Número médio de quartos para casas na mesma cidade.  
- Area Population: A população da cidade onde a casa está localizada.  
- Price: Preço de venda da casa.  
- Address: Endereço da casa.  
Utilize os dados contidos nele para criar um modelo de regressão linear que seja capaz de estimar o preço de venda das casas.  

### Analisando o Dataset

In [ ]:
df2 = pd.read_csv('https://s3-sa-east-1.amazonaws.com/lcpi/7cf57d48-ac3d-4748-9d81-5b4d6677fcff.csv')
df2.head()